In [6]:
import json, requests, time
from bs4 import BeautifulSoup as BS
from collections import Counter

In [7]:
def search_cl_result(query):
    """Search in opinions."""
    OPTIONS = """&type=o&order_by=dateFiled%20desc&stat_Precedential=on"""
    BASE = "https://www.courtlistener.com/api/rest/v3/search/?"

    r = requests.get(BASE+str('q="'+query+'"'+OPTIONS))
    if r.status_code != 200:
        raise Exception('Query not working properly. Try later.')
    data = json.loads(r.content.decode('utf-8'))
    return data

def get_cl_result(opinion_no):
    BASE = "https://www.courtlistener.com/api/rest/v3/opinions/"

    r = requests.get(BASE+str(opinion_no))
    if r.status_code != 200:
        raise Exception('Extraction not done properly.')
    data = json.loads(r.content.decode('utf-8'))
    return data


def extract_text(r_data):
    if not r_data:
        return None
    text = ""
    for each in r_data.keys():
        if each.startswith('html') and each != "html_with_citations":
            text += r_data[each]
    filtered_text = BS(text,'html.parser').get_text()
    return filtered_text
    

In [8]:
def query_american_court(query):
    data = search_cl_result(query)
    total_data = []
    for i in data['results']:
        print(i['id'])
        try:
            total_data.append(get_cl_result(i['id']))
        except Exception as e:
            print(e)
            break
    cleaned_total = []
    for i, item in enumerate(total_data):
        cleaned_total.append(
            {
                "id": item['id'],
                "name": data['results'][i]['caseName'],
                "body": item['plain_text'],
                "node": item
            }
        )
    with open('disease_cases/' + query.replace(' ', '_') + '.json', 'w+') as f:
        json.dump(cleaned_total, f)

In [9]:
diseases = [
    'major depressive disorder',
    'dissociative disorder',
    'Posttraumatic stress disorder',
    'Dissociative identity disorder',
    'pyromania',
    'bipolar disorder',
    'schizophrenia',
    'delusional disorder',
    'obsessive-compulsive disorder',
    'gender dysphoria'
]

In [10]:
for dis in diseases[3:]:
    print(dis)
    query_american_court(dis)
    print()
    time.sleep(120)

Dissociative identity disorder


Exception: Query not working properly. Try later.